In [87]:
import os
import numpy as np
import pandas as pd

In [88]:
df_2023 = pd.read_excel(rf'..\data\raw\raw_2023.xlsx')

In [89]:
df_gross = pd.DataFrame()

In [90]:
## Datetime
df_gross['date'] = df_2023['출고일'].dt.strftime('%Y-%m-%d')
df_gross['year'] = df_2023['출고일'].dt.year
df_gross['month'] = df_2023['출고일'].dt.month
df_gross['day'] = df_2023['출고일'].dt.day

weekday_map = {0: '월', 1: '화', 2: '수', 3: '목', 4: '금', 5: '토', 6: '일'}
df_gross['weekday'] = df_2023['출고일'].dt.weekday.map(weekday_map)

df_gross['hour'] = df_2023['출고일'].dt.hour

In [91]:
### Define Time
def convert_hour_to_text(hour):
    if hour == 0:
        return 'Other'
    elif 1 <= hour < 6:
        return 'Dawn'
    elif 6 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 18:
        return 'Afternoon'
    elif 18 <= hour < 24:
        return 'Evening'
    else:
        return 'Other'

df_gross['time'] = df_gross['hour'].apply(convert_hour_to_text)

In [92]:
## Channel
df_gross['channel'] = df_2023['매출처']

def change_sales_channel(value):
    mapping = {
        '쿠팡': 'wing',
        '제트배송': 'growth',
        '스마트스토어': 'smartstore',
        '자사몰': 'cafe24'
    }
    return mapping.get(value, 'Other')

df_gross['channel'] = df_gross['channel'].apply(change_sales_channel)

In [93]:
df_gross['product'] = df_2023['등록 상품명']
df_gross['option'] = df_2023['등록 옵션명']
df_gross['quantity'] = df_2023['수량']
df_gross['sales'] = df_2023['총 판매가']

In [121]:
df_gross = df_gross.dropna(subset=['quantity','product','sales'])
df_gross = df_gross.loc[~df_gross['product'].str.contains('임직원', na=False)]
df_gross = df_gross.loc[~df_gross['product'].str.contains('보틀', na=False)]
df_gross = df_gross.loc[~df_gross['product'].str.contains('단일', na=False)]
df_gross = df_gross.loc[~df_gross['product'].str.contains('베스트', na=False)]

In [96]:
df_gross['quantity'] = df_gross['quantity'].astype(int)

In [97]:
df_gross['sales']

0        192000
1        167400
2         27900
3         59800
4        160000
          ...  
17828     59800
17829     42000
17830     42000
17831     84000
17832    160000
Name: sales, Length: 16996, dtype: object

In [125]:
df_gross['product'].value_counts()

product
귀리렌틸콩 곤약밥     5427
효소습관 오리지널     4128
철분제           3262
가벼운 습관        1494
슬림케어           564
까만콩 쉐이크        341
아르기닌           327
유산균            252
호박차            250
알로에겔           186
산양유 쉐이크        160
브이티            134
초코 쉐이크         125
닭가슴살           125
따움차             94
현미귀리발효 곤약밥      58
비오틴             33
아임파인            13
차전자피             7
효소 플러스           3
효소 스탠다드          2
아르히 두피 세럼        1
Name: count, dtype: int64

In [123]:
my_dict = {
    '렌틸':'귀리렌틸콩 곤약밥',
    '현미':'현미귀리발효 곤약밥',
    '까만콩':'까만콩 쉐이크',
    '철분':'철분제',
    '프리미엄 효소':'효소습관 오리지널',
    '브이티':'브이티',
    '유산균':'유산균',
    '가르시니아':'가벼운 습관',
    '슬림':'슬림케어',
    '파인':'아임파인',
    '클렌즈':'모로오렌지 클렌즈',
    '스탠다드':'효소 스탠다드',
    '데일리':'효소습관 데일리',
    '따움차':'따움차',
    '알로에':'알로에겔',
    '효소 플러스':'효소 플러스',
    '쉐이크 2종':'까만콩 쉐이크',
    '2박스':'효소습관 오리지널',
    '아르기닌':'아르기닌',
    '호박':'호박차',
    '비오틴':'비오틴',
    '닭가슴살':'닭가슴살',
    '산양유':'산양유 쉐이크',
    '초코 그래놀라':'초코 쉐이크',
    '프로틴':'산양유 쉐이크',
    '모로블러드':'효소습관 모로오렌지',
    '모로오렌지':'효소습관 모로오렌지',
    '차전자피':'차전자피',
    '발포':'가벼운 습관',
    '효소습관':'효소습관 오리지널'
}

In [124]:
for key, value in my_dict.items():
    df_gross.loc[df_gross['product'].str.contains(key), 'product'] = value

In [126]:
df_gross['cat'] = df_gross['product'].apply(
    lambda x: 'enzyme' if '효소' in x else 
              'diet' if '브이티' in x else 
              'meal' if '쉐이크' in x else 
              'meal' if '곤약밥' in x else 
              'healthy' if '철분제' in x else 
              'diet' if '습관' in x else 
              'diet' if '클렌즈' in x else 
              'diet' if '유산균' in x else 
              'healthy' if '알로에겔' in x else 
              'diet' if '슬림' in x else
              'diet' if '아임파인' in x else
              'etc'
)

In [132]:
df_2023 = df_gross

In [133]:
df_2023.to_pickle('df_2023.pkl')